![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Climate Across Provinces

Studying climate patterns across Canadian provinces offers insights into diverse weather behaviors. Analyzing temperature shifts during specific months can reveal important information in regard to impacts on agriculture, biodiversity, and disaster preparedness. Such data helps us understand regional climate variations and make informed decisions for managing resources and planning in the face of changing temperatures.

*Curriculum Connections*

*Investigating Questions*
- Which provinces experience the highest and lowest annual temperatures? What geographical distinctions contribute to these temperature differences?
- Does the variation in elevation significantly affect temperatures across provinces?
- What natural elements, such as proximity to large water bodies or the presence of mountain ranges, can play a role in influencing temperatures across provinces?

***

### Import the Data
The code below will import the Python programming libraries we need to gather and organize the data to answer our question. `▶Run` the code cell below 

In [1]:
import pandas as pd
import plotly.express as px 
import geopandas as gpd
import folium
import re
print("Libraries imported.")

Libraries imported.


### Analysis

Let's begin exploring our dataset containing information about provincial temperatures across Canada. The dataset was obtained from [Kaggle](https://www.kaggle.com/datasets/hemil26/canada-weather), data being scraped from [Wikipedia](https://en.wikipedia.org/wiki/Temperature_in_Canada). 

The dataset when imported is also transformed into a dataframe. You can think of a dataframe as essentially a spreadsheet, where each row represents a record or an observation, while each column represents a different attribute or variable.

In [2]:
climate_data = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Science/ClimateAcrossProvinces/canada_weather.csv')
climate_data

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F))
0,"Alberton, PE",NaN,46°51′00″N 064°01′00″W / 46.85000°N 64.01667°W...,3m (9.8ft),−3.9 (25.0),−12.5 (9.5),23.2 (73.8),14.1 (57.4),9.6 (49.3),1.3 (34.3)
1,"Baker Lake, NU",YBK,64°17′56″N 096°04′40″W / 64.29889°N 96.07778°W...,18.6m (61ft),−27.7 (−17.9),−34.8 (−30.6),17.0 (62.6),6.1 (43.0),−7.3 (18.9),−15.2 (4.6)
2,"Baie-Comeau, QC",YBC,49°08′00″N 068°12′00″W / 49.13333°N 68.20000°W...,22m (72ft),−8.7 (16.3),−19.9 (−3.8),20.9 (69.6),10.3 (50.5),6.6 (43.9),−3.3 (26.1)
3,"Calgary, AB",YYC,51°06′50″N 114°01′13″W / 51.11389°N 114.02028°...,"1,084m (3,556ft)",−0.9 (30.4),−13.2 (8.2),23.2 (73.8),9.8 (49.6),10.8 (51.4),−1.9 (28.6)
4,"Charlottetown, PE",YYG,46°17′19″N 063°07′43″W / 46.28861°N 63.12861°W...,49m (161ft),−3.4 (25.9),−12.1 (10.2),23.3 (73.9),14.1 (57.4),9.9 (49.8),1.3 (34.3)
5,"Churchill, MB",YYQ,58°44′21″N 094°03′59″W / 58.73917°N 94.06639°W...,29m (95ft),−21.9 (−7.4),−30.1 (−22.2),18.0 (64.4),7.3 (45.1),−2.3 (27.9),−10.7 (12.7)
6,"Corner Brook, NL",NaN,48°57′00″N 057°57′00″W / 48.95000°N 57.95000°W...,5m (16ft),−2.7 (27.1),−9.6 (14.7),22.0 (71.6),12.6 (54.7),9.0 (48.2),1.3 (34.3)
7,"Dawson City, YT",YDA,64°02′35″N 139°07′40″W / 64.04306°N 139.12778°...,"370m (1,210ft)",−21.8 (−7.2),−30.1 (−22.2),23.1 (73.6),8.2 (46.8),2.1 (35.8),−10.3 (13.5)
8,"Edmonton, AB",YXD,53°34′24″N 113°31′06″W / 53.57333°N 113.51833°...,"671m (2,201ft)",−6.0 (21.2),−14.8 (5.4),23.1 (73.6),12.3 (54.1),9.3 (48.7),−1.0 (30.2)
9,"Edmundston, NB",NaN,47°20′47″N 068°11′16″W / 47.34639°N 68.18778°W...,163m (535ft),−7.1 (19.2),−18.5 (−1.3),24.7 (76.5),11.5 (52.7),9.5 (49.1),−2.3 (27.9)


We notice that in our dataset, we have *10* different columns. Going from left to right, `Community` refers to the city and province of the climate data, `Weather station` refers to the 3 digit airport code that represents a particular city, `Location` refers to the latitude and longitude coordinates of the particular weather station, and `Elevation` refers how far above each particular station is from sea-level. The rest of the columns seems to refer specific month average temperatures, varying from high to low, or annual average temperatures, which also vary from high to low. 

### Data Cleaning

Now that we've gotten a better sense of what type of information is inside our dataset, let's perform data cleaning.

Data cleaning is like tidying up information to make it useful and accurate. Just like you clean and organize your room, data cleaning helps make data neat and organized.

Imagine you have a bunch of information about your classmates, like their names, ages, and favorite colors. But sometimes, mistakes or errors can happen when collecting this information. For example, someone may have misspelled a name or entered the wrong age for a classmate.

Data cleaning involves finding and fixing these mistakes. It's important because clean data helps us make better decisions and find meaningful patterns.

In our particular situation, we have a couple of things we want to adjust within our dataframe. These include separating provincial codes from cities and adjusting the temperature/elevation columns into only showing numbers for data manipulation later on. We'll go over these steps more in depth in the upcoming cells.

To begin, let's take the provincial codes from each city (which is found at the end of each city) and put the codes into their own column. 

Note: To reduce showing large outputs of **climate_data** every time something is adjusted, we will be using *.sample* to show a random sample of 5 different rows in the data as a means of showing change in the dataframe.

In [3]:
climate_data['Province'] = climate_data.Community.str[-2:]
climate_data.sample(5)

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),Province
2,"Baie-Comeau, QC",YBC,49°08′00″N 068°12′00″W / 49.13333°N 68.20000°W...,22m (72ft),−8.7 (16.3),−19.9 (−3.8),20.9 (69.6),10.3 (50.5),6.6 (43.9),−3.3 (26.1),QC
7,"Dawson City, YT",YDA,64°02′35″N 139°07′40″W / 64.04306°N 139.12778°...,"370m (1,210ft)",−21.8 (−7.2),−30.1 (−22.2),23.1 (73.6),8.2 (46.8),2.1 (35.8),−10.3 (13.5),YT
31,"Summerside, PE",YSU,46°26′20″N 063°49′54″W / 46.43889°N 63.83167°W...,20m (66ft),−3.2 (26.2),−12.1 (10.2),23.8 (74.8),14.6 (58.3),9.9 (49.8),1.6 (34.9),PE
23,"Norman Wells, NT",YVQ,65°16′57″N 126°48′01″W / 65.28250°N 126.80028°...,73m (240ft),−22.2 (−8.0),−29.9 (−21.8),22.5 (72.5),11.5 (52.7),−0.4 (31.3),−9.9 (14.2),NT
29,"Saskatoon, SK",YXE,52°10′00″N 106°43′00″W / 52.16667°N 106.71667°...,"504m (1,654ft)",−10.1 (13.8),−20.7 (−5.3),25.3 (77.5),11.6 (52.9),8.6 (47.5),−3.5 (25.7),SK


Perfect! We now have a new column called `Province` which contains only provincial codes. The next issue we want to tackle is in regard to how our data is presented in columns, particularly the temperature-related columns. 

In these columns, we only want individual temperature values without brackets. This would allow us to define a specific value or float (decimal-point number) to each column entry instead of the entries being considered strings (words). We can fix this by identifying the first *(* bracket and then cutting off the string at this bracket point. Afterwards, we can also replace the string negative sign with an actual negative sign. We then can convert our string into a float as we would only have a string version of a number which can be easily converted by Python. 


In [4]:
def clean_and_convert_to_float(s):
    # replace to actual negative float symbol
    cleaned_value = s.split('(')[0].strip().replace('−', '-')
    try:
        return float(cleaned_value)
    except ValueError:
        return None  

In [5]:
columns_to_clean = [
    'Annual(Avg. high °C (°F))',
    'January(Avg. high °C (°F))',
    'January(Avg. low °C (°F))',
    'July(Avg. high °C (°F))',
    'July(Avg. low °C (°F))',
    'Annual(Avg. low °C (°F))'
]

for column in columns_to_clean:
    climate_data[column] = climate_data[column].apply(clean_and_convert_to_float)
climate_data.sample(5)

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),Province
0,"Alberton, PE",NaN,46°51′00″N 064°01′00″W / 46.85000°N 64.01667°W...,3m (9.8ft),-3.9,-12.5,23.2,14.1,9.6,1.3,PE
11,"Fredericton, NB",YFC,45°52′20″N 066°31′40″W / 45.87222°N 66.52778°W...,21m (69ft),-3.8,-15.0,25.5,13.0,11.4,-0.2,NB
9,"Edmundston, NB",NaN,47°20′47″N 068°11′16″W / 47.34639°N 68.18778°W...,163m (535ft),-7.1,-18.5,24.7,11.5,9.5,-2.3,NB
3,"Calgary, AB",YYC,51°06′50″N 114°01′13″W / 51.11389°N 114.02028°...,"1,084m (3,556ft)",-0.9,-13.2,23.2,9.8,10.8,-1.9,AB
15,"Iqaluit, NU",YFB,63°45′00″N 068°33′00″W / 63.75000°N 68.55000°W...,34m (112ft),-22.8,-30.9,12.3,4.1,-5.6,-13.1,NU


Now our values are only single decimal-point numbers, alongside negative numbers. The final thing we need to do is fix the `Elevation` column. Similarly to our temperature columns, we want to only have a single decimal-point number in these column entries for easier data manipulation later on. This is achieved similarly where we find the first instance of an *m* and then cut the string off from that point. We also have cases of numbers having commas in them such as 1,084m. To get around this, whenever we see a *,*, we can replace it with an empty space.  

In [6]:
def fix_elevation(text):
    new_elevation = text.split('m')[0].replace(',', '').strip()
    try:
        return float(new_elevation)
    except ValueError:
        return None

In [7]:
climate_data['Elevation'] = climate_data['Elevation'].apply(fix_elevation)
climate_data.sample(5)

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),Province
12,"Halifax, NS",YHZ,44°52′48″N 063°30′00″W / 44.88000°N 63.50000°W...,145.0,-1.3,-10.4,23.8,13.7,11.3,1.9,NS
33,"Thompson, MB",YTH,55°48′12″N 097°51′45″W / 55.80333°N 97.86250°W...,224.0,-18.3,-29.3,23.1,9.1,3.4,-9.1,MB
16,"Kamloops, BC",YKA,50°42′08″N 120°26′31″W / 50.70222°N 120.44194°...,345.3,0.4,-5.9,28.9,14.2,14.8,3.7,BC
32,"Sydney, NS",YQY,46°10′00″N 060°02′53″W / 46.16667°N 60.04806°W...,62.0,-1.1,-9.6,23.1,12.6,10.3,1.4,NS
24,"Ottawa, ON",YOW,45°19′21″N 075°40′09″W / 45.32250°N 75.66917°W...,114.0,-5.8,-14.8,26.5,15.5,11.3,1.4,ON


Perfect! Now that we're done cleaning our data, we can move onto analysis and visualizing our data. First, since we have access to months, January and July, that are correlated with *winter* and *summer* respectively, let's find which city/province contains the coldest and hottest temperatures in our dataframe.

In [8]:
lowest_jan_temp_data = climate_data[climate_data['January(Avg. low °C (°F))'] == climate_data['January(Avg. low °C (°F))'].min()]
highest_july_temp_data = climate_data[climate_data['July(Avg. high °C (°F))'] == climate_data['July(Avg. high °C (°F))'].max()]

location_jan = lowest_jan_temp_data['Community'].to_string(index=False)
location_july = highest_july_temp_data['Community'].to_string(index=False)

jan_temp = lowest_jan_temp_data['January(Avg. low °C (°F))'].to_string(index=False)
july_temp = highest_july_temp_data['July(Avg. high °C (°F))'].to_string(index=False)

print(f"The lowest temperature recorded in January in the dataset was in {location_jan}, with a recorded temperature of {jan_temp}°C")
print(f"The highest temperature recorded in July in the dataset was in {location_july}, with a recorded temperature of {july_temp}°C")

The lowest temperature recorded in January in the dataset was in Resolute, NU, with a recorded temperature of -35.3°C
The highest temperature recorded in July in the dataset was in Kamloops, BC, with a recorded temperature of 28.9°C


It appears that the coldest recorded temperature in January was recorded in Nunavut, and the highest recorded temperature in July was recorded in British Columbia. 

This makes sense due to the geographical and climatic differences between Nunavut and British Columbia. Nunavut is a territory located in northern Canada, characterized by its high latitude and Arctic climate. In January, during the heart of winter, Nunavut experiences extreme cold temperatures due to its proximity to the Arctic Circle and its exposure to polar air masses.

On the other hand, British Columbia is a province situated on the west coast of Canada. It benefits from the moderating influence of the Pacific Ocean, which tends to keep its coastal regions milder in comparison to more inland areas. In July, during the summer months, British Columbia can experience higher temperatures due to the ocean's ability to store and release heat slowly, leading to warmer conditions along its coastline.

However, considering the averages of the highest and lowest temperatures for each month might provide a more comprehensive understanding of the overall climate in these regions. Let's do this by taking the average between both the January and July temperature columns. 

In [9]:
climate_data['January Average'] = climate_data[['January(Avg. low °C (°F))', 'January(Avg. high °C (°F))']].mean(axis=1)
climate_data['July Average'] = climate_data[['July(Avg. low °C (°F))', 'July(Avg. high °C (°F))']].mean(axis=1)
climate_data.sample(5)

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),Province,January Average,July Average
9,"Edmundston, NB",NaN,47°20′47″N 068°11′16″W / 47.34639°N 68.18778°W...,163.0,-7.1,-18.5,24.7,11.5,9.5,-2.3,NB,-12.80,18.1
4,"Charlottetown, PE",YYG,46°17′19″N 063°07′43″W / 46.28861°N 63.12861°W...,49.0,-3.4,-12.1,23.3,14.1,9.9,1.3,PE,-7.75,18.7
39,"Windsor, ON",YQG,42°16′32″N 82°57′20″W / 42.27556°N 82.95556°W ...,190.0,-0.3,-7.3,28.1,17.9,14.4,5.4,ON,-3.80,23.0
23,"Norman Wells, NT",YVQ,65°16′57″N 126°48′01″W / 65.28250°N 126.80028°...,73.0,-22.2,-29.9,22.5,11.5,-0.4,-9.9,NT,-26.05,17.0
6,"Corner Brook, NL",NaN,48°57′00″N 057°57′00″W / 48.95000°N 57.95000°W...,5.0,-2.7,-9.6,22.0,12.6,9.0,1.3,NL,-6.15,17.3


In [21]:
avg_jan_temp = climate_data[climate_data['January Average'] == climate_data['January Average'].min()]
avg_july_temp = climate_data[climate_data['July Average'] == climate_data['July Average'].max()]

location_jan = avg_jan_temp['Community'].to_string(index=False)
location_july = avg_july_temp['Community'].to_string(index=False)

jan_temp = avg_jan_temp['January Average'].to_string(index=False)
july_temp = avg_july_temp['July Average'].to_string(index=False)

print(f"The lowest averagetemperature recorded in January in the dataset was in {location_jan}, with a recorded temperature of {jan_temp}°C")
print(f"The highest average temperature recorded in July in the dataset was in {location_july}, with a recorded temperature of {july_temp}°C")

The lowest averagetemperature recorded in January in the dataset was in Resolute, NU, with a recorded temperature of -31.95°C
The highest average temperature recorded in July in the dataset was in Windsor, ON, with a recorded temperature of 23.0°C


Let's also find the overall highest/lowest temperatures recorded in our dataframe and see if it differs to our findings in January and July. 

In [10]:
overall_highest_temp = climate_data[climate_data['Annual(Avg. high °C (°F))'] == climate_data['Annual(Avg. high °C (°F))'].max()]
overall_lowest_temp = climate_data[climate_data['Annual(Avg. low °C (°F))'] == climate_data['Annual(Avg. low °C (°F))'].min()]
highest_overall_loc = overall_highest_temp['Community'].to_string(index=False)
lowest_overall_loc = overall_lowest_temp['Community'].to_string(index=False)
highest_temp = overall_highest_temp['Annual(Avg. high °C (°F))'].to_string(index=False)
lowest_temp = overall_lowest_temp['Annual(Avg. high °C (°F))'].to_string(index=False)

print(f"The ovearll lowest annual temperature recorded in the dataset was in {lowest_overall_loc}, with a recorded temperature of {lowest_temp}°C")
print(f"The overall highest annual temperature recorded in the dataset was in {highest_overall_loc}, with a recorded temperature of {highest_temp}°C")

The ovearll lowest annual temperature recorded in the dataset was in Resolute, NU, with a recorded temperature of -12.7°C
The overall highest annual temperature recorded in the dataset was in Kamloops, BC, with a recorded temperature of 14.8°C


Looking at the output, it appears there are no differences compared to our output above. This generally makes sense as the province that records the coldest temperature in January, a month characterized by winter, naturally tends to have a climate that sustains cold conditions throughout the year. In contrast, the province boasting the highest July temperatures, a month representing the peak of summer, naturally maintains a warmer climate on an annual basis. 

In [11]:
elevation_fig = px.scatter(climate_data, x='Elevation', y='Annual(Avg. high °C (°F))', color='Province').show()

In [12]:
elevation_fig = px.scatter(climate_data, x='Elevation', y='Annual(Avg. low °C (°F))', color='Province').show()

In [13]:
climate_data

,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),Province,January Average,July Average
0,"Alberton, PE",NaN,46°51′00″N 064°01′00″W / 46.85000°N 64.01667°W...,3.0,-3.9,-12.5,23.2,14.1,9.6,1.3,PE,-8.20,18.65
1,"Baker Lake, NU",YBK,64°17′56″N 096°04′40″W / 64.29889°N 96.07778°W...,18.6,-27.7,-34.8,17.0,6.1,-7.3,-15.2,NU,-31.25,11.55
2,"Baie-Comeau, QC",YBC,49°08′00″N 068°12′00″W / 49.13333°N 68.20000°W...,22.0,-8.7,-19.9,20.9,10.3,6.6,-3.3,QC,-14.30,15.60
3,"Calgary, AB",YYC,51°06′50″N 114°01′13″W / 51.11389°N 114.02028°...,1084.0,-0.9,-13.2,23.2,9.8,10.8,-1.9,AB,-7.05,16.50
4,"Charlottetown, PE",YYG,46°17′19″N 063°07′43″W / 46.28861°N 63.12861°W...,49.0,-3.4,-12.1,23.3,14.1,9.9,1.3,PE,-7.75,18.70
5,"Churchill, MB",YYQ,58°44′21″N 094°03′59″W / 58.73917°N 94.06639°W...,29.0,-21.9,-30.1,18.0,7.3,-2.3,-10.7,MB,-26.00,12.65
6,"Corner Brook, NL",NaN,48°57′00″N 057°57′00″W / 48.95000°N 57.95000°W...,5.0,-2.7,-9.6,22.0,12.6,9.0,1.3,NL,-6.15,17.30
7,"Dawson City, YT",YDA,64°02′35″N 139°07′40″W / 64.04306°N 139.12778°...,370.0,-21.8,-30.1,23.1,8.2,2.1,-10.3,YT,-25.95,15.65
8,"Edmonton, AB",YXD,53°34′24″N 113°31′06″W / 53.57333°N 113.51833°...,671.0,-6.0,-14.8,23.1,12.3,9.3,-1.0,AB,-10.40,17.70
9,"Edmundston, NB",NaN,47°20′47″N 068°11′16″W / 47.34639°N 68.18778°W...,163.0,-7.1,-18.5,24.7,11.5,9.5,-2.3,NB,-12.80,18.10


In [14]:
prov_data = gpd.read_file('https://raw.githubusercontent.com/callysto/data-files/main/Science/ClimateAcrossProvinces/geopandas.geojson')
prov_data

,geo_point_2d,year,prov_code,prov_name_en,prov_area_code,prov_type,prov_name_fr,geometry
0,"{'lon': -114.51101438986483, 'lat': 55.1689029...",2021,[48],[Alberta],CAN,province,Alberta,"POLYGON ((-110.00502 48.99970, -110.00449 49.5..."
1,"{'lon': -97.43375328505664, 'lat': 54.93018410...",2021,[46],[Manitoba],CAN,province,Manitoba,"POLYGON ((-94.82808 60.00000, -95.32500 60.000..."
2,"{'lon': -135.51007960983821, 'lat': 63.6304438...",2021,[60],[Yukon],CAN,territory / territoire,Yukon,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478..."
3,"{'lon': -60.51575069823122, 'lat': 52.88843621...",2021,[10],[Newfoundland and Labrador],CAN,province,Terre-Neuve-et-Labrador,"MULTIPOLYGON (((-53.63678 48.87151, -53.67894 ..."
4,"{'lon': -66.37667726184117, 'lat': 46.61898879...",2021,[13],[New Brunswick],CAN,province,Nouveau-Brunswick,"MULTIPOLYGON (((-66.93908 45.02470, -67.01600 ..."
5,"{'lon': -105.89075902236681, 'lat': 54.4167141...",2021,[47],[Saskatchewan],CAN,province,Saskatchewan,"POLYGON ((-102.00000 60.00000, -102.75000 60.0..."
6,"{'lon': -63.32450309629852, 'lat': 45.15131558...",2021,[12],[Nova Scotia],CAN,province,Nouvelle-Écosse,"MULTIPOLYGON (((-65.60612 43.51377, -65.65127 ..."
7,"{'lon': -118.98262958354398, 'lat': 66.3689231...",2021,[61],[Northwest Territories],CAN,territory / territoire,Territoires du Nord-Ouest,"MULTIPOLYGON (((-109.83296 75.93393, -110.0324..."
8,"{'lon': -63.24228135586117, 'lat': 46.38954858...",2021,[11],[Prince Edward Island],CAN,province,Île-du-Prince-Édouard,"POLYGON ((-61.98606 46.46286, -62.14666 46.484..."
9,"{'lon': -88.8416338799438, 'lat': 71.041389444...",2021,[62],[Nunavut],CAN,territory / territoire,Nunavut,"MULTIPOLYGON (((-106.67059 73.67679, -106.8352..."


In [15]:
prov_data.drop(columns=['prov_name_en', 'geo_point_2d', 'prov_area_code', 'year'],inplace=True)

In [16]:
prov_data.prov_name_fr.replace(
    {
        'Alberta': 'AB',
        'Manitoba': 'MB',
        'Yukon': 'YT',
        'Terre-Neuve-et-Labrador': 'NL',
        'Nouveau-Brunswick': 'NB',
        'Saskatchewan': 'SK',
        'Nouvelle-Écosse': 'NS',
        'Territoires du Nord-Ouest': 'NT',
        'Île-du-Prince-Édouard': 'PE',
        'Nunavut': 'NU',
        'Québec': 'QC',
        'Ontario': 'ON',
        'Colombie-Britannique': 'BC'
    },
    inplace=True
)

prov_data.rename(columns={'prov_name_fr': "Province"}, inplace=True)  
prov_data

,prov_code,prov_type,Province,geometry
0,[48],province,AB,"POLYGON ((-110.00502 48.99970, -110.00449 49.5..."
1,[46],province,MB,"POLYGON ((-94.82808 60.00000, -95.32500 60.000..."
2,[60],territory / territoire,YT,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478..."
3,[10],province,NL,"MULTIPOLYGON (((-53.63678 48.87151, -53.67894 ..."
4,[13],province,NB,"MULTIPOLYGON (((-66.93908 45.02470, -67.01600 ..."
5,[47],province,SK,"POLYGON ((-102.00000 60.00000, -102.75000 60.0..."
6,[12],province,NS,"MULTIPOLYGON (((-65.60612 43.51377, -65.65127 ..."
7,[61],territory / territoire,NT,"MULTIPOLYGON (((-109.83296 75.93393, -110.0324..."
8,[11],province,PE,"POLYGON ((-61.98606 46.46286, -62.14666 46.484..."
9,[62],territory / territoire,NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.8352..."


In [17]:
data_w_climate = prov_data.merge(climate_data, left_on='Province', right_on='Province', how='left')
data_w_climate

,prov_code,prov_type,Province,geometry,Community,Weather station,Location,Elevation,January(Avg. high °C (°F)),January(Avg. low °C (°F)),July(Avg. high °C (°F)),July(Avg. low °C (°F)),Annual(Avg. high °C (°F)),Annual(Avg. low °C (°F)),January Average,July Average
0,[48],province,AB,"POLYGON ((-110.00502 48.99970, -110.00449 49.5...","Calgary, AB",YYC,51°06′50″N 114°01′13″W / 51.11389°N 114.02028°...,1084.0,-0.9,-13.2,23.2,9.8,10.8,-1.9,-7.05,16.50
1,[48],province,AB,"POLYGON ((-110.00502 48.99970, -110.00449 49.5...","Edmonton, AB",YXD,53°34′24″N 113°31′06″W / 53.57333°N 113.51833°...,671.0,-6.0,-14.8,23.1,12.3,9.3,-1.0,-10.40,17.70
2,[48],province,AB,"POLYGON ((-110.00502 48.99970, -110.00449 49.5...","High Level, AB",YOJ,58°37′17″N 117°09′53″W / 58.62139°N 117.16472°...,338.0,-15.0,-25.8,23.0,9.9,5.2,-7.2,-20.40,16.45
3,[46],province,MB,"POLYGON ((-94.82808 60.00000, -95.32500 60.000...","Churchill, MB",YYQ,58°44′21″N 094°03′59″W / 58.73917°N 94.06639°W...,29.0,-21.9,-30.1,18.0,7.3,-2.3,-10.7,-26.00,12.65
4,[46],province,MB,"POLYGON ((-94.82808 60.00000, -95.32500 60.000...","Thompson, MB",YTH,55°48′12″N 097°51′45″W / 55.80333°N 97.86250°W...,224.0,-18.3,-29.3,23.1,9.1,3.4,-9.1,-23.80,16.10
5,[46],province,MB,"POLYGON ((-94.82808 60.00000, -95.32500 60.000...","Winnipeg, MB",YWG,49°55′00″N 097°14′00″W / 49.91667°N 97.23333°W...,239.0,-11.3,-21.4,25.9,13.5,8.7,-2.7,-16.35,19.70
6,[60],territory / territoire,YT,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478...","Dawson City, YT",YDA,64°02′35″N 139°07′40″W / 64.04306°N 139.12778°...,370.0,-21.8,-30.1,23.1,8.2,2.1,-10.3,-25.95,15.65
7,[60],territory / territoire,YT,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478...","Mayo, YT",YMA,63°37′00″N 135°52′00″W / 63.61667°N 135.86667°...,504.0,-18.0,-28.2,22.8,9.4,3.4,-8.2,-23.10,16.10
8,[60],territory / territoire,YT,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478...","Whitehorse, YT",YXY,60°42′34″N 135°04′08″W / 60.70944°N 135.06889°...,706.0,-11.0,-19.2,20.6,8.0,5.1,-5.2,-15.10,14.30
9,[10],province,NL,"MULTIPOLYGON (((-53.63678 48.87151, -53.67894 ...","Corner Brook, NL",NaN,48°57′00″N 057°57′00″W / 48.95000°N 57.95000°W...,5.0,-2.7,-9.6,22.0,12.6,9.0,1.3,-6.15,17.30


In [18]:
agg_data = data_w_climate.groupby("Province")["Annual(Avg. high °C (°F))"].mean().reset_index()
merged_data = prov_data.merge(agg_data, on="Province")
merged_data

,prov_code,prov_type,Province,geometry,Annual(Avg. high °C (°F))
0,[48],province,AB,"POLYGON ((-110.00502 48.99970, -110.00449 49.5...",8.433333
1,[46],province,MB,"POLYGON ((-94.82808 60.00000, -95.32500 60.000...",3.266667
2,[60],territory / territoire,YT,"MULTIPOLYGON (((-139.13388 69.63737, -139.2478...",3.533333
3,[10],province,NL,"MULTIPOLYGON (((-53.63678 48.87151, -53.67894 ...",6.566667
4,[13],province,NB,"MULTIPOLYGON (((-66.93908 45.02470, -67.01600 ...",10.533333
5,[47],province,SK,"POLYGON ((-102.00000 60.00000, -102.75000 60.0...",7.933333
6,[12],province,NS,"MULTIPOLYGON (((-65.60612 43.51377, -65.65127 ...",10.900000
7,[61],territory / territoire,NT,"MULTIPOLYGON (((-109.83296 75.93393, -110.0324...",-1.300000
8,[11],province,PE,"POLYGON ((-61.98606 46.46286, -62.14666 46.484...",9.800000
9,[62],territory / territoire,NU,"MULTIPOLYGON (((-106.67059 73.67679, -106.8352...",-7.925000


In [19]:
# Initialize the map and store it in an m object
highest_temp_map = folium.Map(location=[50, -65], zoom_start=3)

folium.Choropleth(
    geo_data=prov_data,
    data=merged_data,
    name="choropleth",
    columns=["Province", "Annual(Avg. high °C (°F))"],
    fill_color="YlOrRd",
    key_on="feature.properties.Province",
    legend_name="Average Highest Temperature (Annual)",
).add_to(highest_temp_map)

folium.LayerControl().add_to(highest_temp_map)
highest_temp_map

In [20]:
agg_data = data_w_climate.groupby("Province")["Annual(Avg. low °C (°F))"].mean().reset_index()
merged_data_low_temp = prov_data.merge(agg_data, on="Province")

# Initialize the map and store it in an m object
lowest_temp_map = folium.Map(location=[50, -65], zoom_start=3)

folium.Choropleth(
    geo_data=prov_data,
    data=merged_data_low_temp,
    name="choropleth",
    columns=["Province", "Annual(Avg. low °C (°F))"],
    fill_color="PuBu",
    key_on="feature.properties.Province",
    legend_name="Average Lowest Temperature (Annual)",
).add_to(lowest_temp_map)

folium.LayerControl().add_to(lowest_temp_map)
lowest_temp_map